In [6]:
import torch
import torch.utils.data as utils
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
import random
from torch.autograd import Variable, grad
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math

In [7]:
class FilterLinear(nn.Module):
    def __init__(self, in_features, out_features, filter_square_matrix, bias=True):
        '''
        filter_square_matrix : filter square matrix, whose each elements is 0 or 1.
        '''
        super(FilterLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        use_gpu = torch.cuda.is_available()
        self.filter_square_matrix = None
        if use_gpu:
            self.filter_square_matrix = Variable(filter_square_matrix.cuda(), requires_grad=False)
        else:
            self.filter_square_matrix = Variable(filter_square_matrix, requires_grad=False)
        
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
#         print(self.weight.data)
#         print(self.bias.data)

    def forward(self, input):
#         print(self.filter_square_matrix.mul(self.weight))
        return F.linear(input, self.filter_square_matrix.mul(self.weight), self.bias)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'

In [8]:
import os 
os.chdir(os.path.pardir)
# load data from file 
import numpy as np 
save_file_name = ['fea_seq.npy', 'last_observation_seq.npy', 'label_seq.npy', 'masking_seq.npy',
                   'delta_seq.npy', 'train_valid_test_split.npy']
save_folder = 'data/raw/pol_temp_rh'
saved_arrays = []
for file_name in save_file_name:
    saved_arrays.append(np.load(os.path.join(save_folder, file_name)))
[fea_seq, last_observation_seq, label_seq, masking_seq, delta_seq, train_valid_test_split] = saved_arrays

# train-test-split 
train_index = [k for k in range(train_valid_test_split[0])]
dev_index = [k for k in range(train_valid_test_split[0], 
                               train_valid_test_split[0] + train_valid_test_split[1])]
test_index = [k for k in range(train_valid_test_split[0] + train_valid_test_split[1],
              train_valid_test_split[0] + train_valid_test_split[1] + train_valid_test_split[2])]

def get_array_by_index_range(nparray_list, label_array, index_range):
    '''
    nparray_list: list of nparrays to select according to index range 
    label_array: select the labels from label array
    '''
    # get non-na index
    non_na_index = []
    for index in index_range:
        if not np.isnan(label_array[index]):
            non_na_index.append(index)
    
    return [k[non_na_index] for k in nparray_list], label_array[non_na_index].reshape(-1)

In [9]:
# normalize delta
# we can normlize this because it's know features 
delta_seq = (delta_seq - np.mean(delta_seq)) / np.std(delta_seq) 

In [10]:
# split set to train, test and dev sets 
# train set
[fea_train, last_train, masking_train, delta_train], label_train =  get_array_by_index_range([fea_seq,last_observation_seq, masking_seq, delta_seq
                                                                 ], label_seq, train_index)
# dev set 
[fea_dev, last_dev, masking_dev, delta_dev], label_dev =  get_array_by_index_range([fea_seq, last_observation_seq, masking_seq, delta_seq
                                                           ], label_seq, dev_index)
# test set 
[fea_test, last_test, masking_test, delta_test], label_test =  get_array_by_index_range([fea_seq, last_observation_seq, masking_seq, delta_seq
                                                              ], label_seq, test_index)

def normalize_feature(fea_train, array_list):
    """
    array_list: [fea_dev, fea_test, last_train, last_dev, last_test] to normalize 
    """
    train_mean = np.nanmean(fea_train, axis=0)
    train_std = np.nanstd(fea_train, axis=0)
    def norm_arr(nparr):
        return(nparr - train_mean)/train_std
    return (norm_arr(fea_train), [norm_arr(k) for k in array_list])

fea_train, [fea_dev, fea_test, last_train, last_dev, last_test] = normalize_feature(fea_train,
                                                                                   [fea_dev, fea_test, 
                                                                                    last_train, last_dev,
                                                                                    last_test])


In [11]:
# record mean after normalization 
x_mean_aft_nor = np.nanmean(fea_train, axis=0)

In [12]:
# get dataset for grud 
def dataset_aggregation(feature_array, last_obsv, mask, delta):
    # expand dimension of array
    def expd(arr):
        return np.expand_dims(arr, axis=1)
    return np.concatenate((expd(feature_array), expd(last_obsv), expd(mask), expd(delta)), axis = 1)

In [13]:
# dataset_aggregation for train, dev, test 
# train_aggr = dataset_aggregation(fea_train, last_train, masking_train, delta_train)
train_aggr = dataset_aggregation(last_train, last_train, masking_train, delta_train)
# dev_aggr = dataset_aggregation(fea_dev, last_dev, masking_dev, delta_dev)
# test_aggr = dataset_aggregation(fea_test, last_test, masking_test, delta_test)
dev_aggr = dataset_aggregation(last_dev, last_dev, masking_dev, delta_dev)
test_aggr = dataset_aggregation(last_test, last_test, masking_test, delta_test)

In [14]:
train_aggr.shape

(664, 4, 7, 5)

In [15]:
def train_mfn(X_train, y_train, X_valid, y_valid, X_test, y_test, configs, X_mean):
#     p = np.random.permutation(X_train.shape[0])
    # no shuffle, keep original order 
    # swap axes for back propagation 
#     def swap_axes(nparr):
#         return nparr.swapaxes(0,1)
#     X_train = swap_axes(X_train)
#     X_valid = swap_axes(X_valid)
#     X_test = swap_axes(X_test)
    
    # model parameters 
    input_size = X_train.shape[3]
    h = 32
    t = X_train.shape[2]
    output_dim = 1
    dropout = 0.5

#     d = X_train.shape[2]
#     h = 128
#     t = X_train.shape[0]
#     output_dim = 1
#     dropout = 0.5

    [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig] = configs

    
    #model = EFLSTM(d,h,output_dim,dropout)
    model = MFN(config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig, X_mean)

    optimizer = optim.Adam(model.parameters(),lr=config["lr"])
    #optimizer = optim.SGD(model.parameters(),lr=config["lr"],momentum=config["momentum"])

    # optimizer = optim.SGD([
    #                 {'params':model.lstm_l.parameters(), 'lr':config["lr"]},
    #                 {'params':model.classifier.parameters(), 'lr':config["lr"]}
    #             ], momentum=0.9)

    criterion = nn.MSELoss()
    device = torch.device('cpu')
    model = model.to(device)
    criterion = criterion.to(device)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=10, factor=0.5, verbose=True)
    
#     criterion = nn.L1Loss()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = model.to(device)
#     criterion = criterion.to(device)
#     scheduler = ReduceLROnPlateau(optimizer,mode='min',patience=100,factor=0.5,verbose=True)

    def train(model, batchsize, X_train, y_train, optimizer, criterion):
        epoch_loss = 0
        model.train()
        total_n = X_train.shape[0]
        num_batches = math.ceil(total_n / batchsize)
        for batch in range(num_batches):
            start = batch*batchsize
            end = (batch+1)*batchsize
            optimizer.zero_grad()
            batch_X = torch.Tensor(X_train[start:end,:])
            batch_y = torch.Tensor(y_train[start:end])
            predictions = model.forward(batch_X).squeeze(1)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / num_batches

    def evaluate(model, X_valid, y_valid, criterion):
        epoch_loss = 0
        model.eval()
        with torch.no_grad():
            batch_X = torch.Tensor(X_valid)
            batch_y = torch.Tensor(y_valid)
            predictions = model.forward(batch_X).squeeze(1)
            epoch_loss = criterion(predictions, batch_y).item()
        return epoch_loss

    def predict(model, X_test):
        epoch_loss = 0
        model.eval()
        with torch.no_grad():
            batch_X = torch.Tensor(X_test)
            predictions = model.forward(batch_X).squeeze(1)
            predictions = predictions.cpu().data.numpy()
        return predictions

    best_valid = 999999.0
    rand = random.randint(0,100000)
    print('epoch train_loss valid_loss')
    for epoch in range(config["num_epochs"]):
        train_loss = train(model, config["batchsize"], X_train, y_train, optimizer, criterion)
        valid_loss = evaluate(model, X_valid, y_valid, criterion)
        scheduler.step(valid_loss)
        if valid_loss <= best_valid:
            # save model
            best_valid = valid_loss
            print(epoch, train_loss, valid_loss, 'saving model')
            torch.save(model, 'models/temp_models/mfn_%d.pt' %rand)
        else:
            print(epoch, train_loss, valid_loss)

#     print 'model number is:', rand
    model = torch.load('models/temp_models/mfn_%d.pt' %rand)

    predictions = predict(model, X_test)
    mae = np.mean(np.absolute(predictions-y_test))
    print("mae: ", mae)
    mse = np.mean((predictions - y_test)**2)
    print("mse: ", mse)

In [16]:
class GRUD(nn.Module):
    def __init__(self, input_size, hidden_size, X_mean, output_last = True, dropout=0):
        """
        Recurrent Neural Networks for Multivariate Times Series with Missing Values
        GRU-D: GRU exploit two representations of informative missingness patterns, i.e., masking and time interval.
        cell_size is the size of cell_state.
        
        GRU-D:
            input_size: variable dimension of each time
            hidden_size: dimension of hidden_state
            mask_size: dimension of masking vector
            X_mean: the mean of the historical input data
        """
        super(GRUD, self).__init__()
        
        self.hidden_size = hidden_size
        self.delta_size = input_size
        self.mask_size = input_size
        
        self.identity = torch.eye(input_size)
        self.zeros = Variable(torch.zeros(input_size))
        
        self.zl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size)
        self.rl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size)
        self.hl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size)
        
        self.gamma_x_l = FilterLinear(self.delta_size, self.delta_size, self.identity)
        
#         self.gamma_h_l = nn.Linear(self.delta_size, self.delta_size)
        self.gamma_h_l = nn.Linear(self.delta_size, hidden_size)
        
        self.map_hidden_gamma = nn.Linear(hidden_size, 1)
        
        self.output_last = output_last
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)
        
#     def forward(self, x, x_last_obsv, x_mean, h, mask, delta):
    # use input to feed the four dimensions 
    def forward(self, dataset, X_mean, h):
        """
        dataset: matrix shape (n * c * d)
                    n: batchsize; c: class; d number of dims
        """
        x_mean = Variable(torch.Tensor(X_mean))
#         print("dataset size")
#         print(dataset.size())
        batch_size = dataset.size(0)
        dim_size = dataset.size(2)
    
        x = dataset[:,0,:]
        x_last_obsv = dataset[:,1,:]
        mask = dataset[:,2,:]
        delta = dataset[:,3,:]
        
        delta_x = torch.exp(-torch.max(self.zeros, self.gamma_x_l(delta)))
        
        delta_h = F.sigmoid(self.map_hidden_gamma(F.relu(self.gamma_h_l(delta))))
        
        x = mask * x + (1 - mask) * (delta_x * x_last_obsv + (1 - delta_x) * x_mean)
        h = delta_h * h 
    
#         print("combined size")
#         print(x.size())
#         print(h.size())
#         print(mask.size())
        combined = torch.cat((x, h, mask), 1)
        z = torch.sigmoid(self.zl(combined))
        r = torch.sigmoid(self.rl(combined))
        combined_r = torch.cat((x, r * h, mask), 1)
        h_tilde = torch.tanh(self.hl(combined_r))
        h = (1 - z) * h + z * h_tilde
        
        return h
     

In [17]:
class MFN(nn.Module):
    def __init__(self,config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig, X_mean):
        super(MFN, self).__init__()
        [self.d_l,self.d_a] = config["input_dims"]
        [self.dh_l,self.dh_a] = config["h_dims"]
        total_h_dim = self.dh_l+self.dh_a
        
        self.mem_dim = config["memsize"]
        window_dim = config["windowsize"]
        output_dim = 1
        attInShape = total_h_dim*window_dim
        gammaInShape = attInShape+self.mem_dim
        final_out = total_h_dim+self.mem_dim
        h_att1 = NN1Config["shapes"]
        h_att2 = NN2Config["shapes"]
        h_gamma1 = gamma1Config["shapes"]
        h_gamma2 = gamma2Config["shapes"]
        h_out = outConfig["shapes"]
        att1_dropout = NN1Config["drop"]
        att2_dropout = NN2Config["drop"]
        gamma1_dropout = gamma1Config["drop"]
        gamma2_dropout = gamma2Config["drop"]
        out_dropout = outConfig["drop"]
    
        # use grud cell instead of LSTM Cell
#         (self, input_size, hidden_size, X_mean, output_last = True, dropout=0):
        self.gru_l = GRUD(self.d_l, self.dh_l, X_mean)
        self.gru_a = GRUD(self.d_a, self.dh_a, X_mean)

        self.att1_fc1 = nn.Linear(attInShape, h_att1)
        self.att1_fc2 = nn.Linear(h_att1, attInShape)
        self.att1_dropout = nn.Dropout(att1_dropout)

        self.att2_fc1 = nn.Linear(attInShape, h_att2)
        self.att2_fc2 = nn.Linear(h_att2, self.mem_dim)
        self.att2_dropout = nn.Dropout(att2_dropout)

        self.gamma1_fc1 = nn.Linear(gammaInShape, h_gamma1)
        self.gamma1_fc2 = nn.Linear(h_gamma1, self.mem_dim)
        self.gamma1_dropout = nn.Dropout(gamma1_dropout)

#         self.gamma2_fc1 = nn.Linear(gammaInShape, h_gamma2)
#         self.gamma2_fc2 = nn.Linear(h_gamma2, self.mem_dim)
#         self.gamma2_dropout = nn.Dropout(gamma2_dropout)

        self.out_fc1 = nn.Linear(final_out, h_out)
        self.out_fc2 = nn.Linear(h_out, output_dim)
        self.out_dropout = nn.Dropout(out_dropout)
        
        ## add the part for grud missing imputation 
        self.x_mean = X_mean 
        self.hidden_size = self.dh_l
        
        
        ## 
        self.gamma_decay = nn.Linear(self.d_l+self.d_a, self.hidden_size)
        self.map_gamma_decay = nn.Linear(self.hidden_size, 1)
        
        
    def squeeze_d2(self, matrix):
        return torch.squeeze(matrix, dim=2)
        
    def forward(self,x):
        x_l = x[:,:,:,:self.d_l]
        x_a = x[:,:,:,self.d_l:self.d_l+self.d_a]
        # x is n x c x t x d
        # seperate x_mean_l and x_mean_a 
        x_mean_l = self.x_mean[:,:self.d_l]
        x_mean_a = self.x_mean[:,self.d_l:self.d_l+self.d_a]
        
        n = x.shape[0]
        t = x.shape[1]
        self.h_l = self.initHidden(n)
        self.h_a = self.initHidden(n)
        
        self.mem = torch.zeros(n, self.mem_dim)
        all_h_ls = []
        all_h_as = []

        all_mems = []
        
        for i in range(t):
            # prev time step
            prev_h_l = self.h_l
            prev_h_a = self.h_a
#             print("x_l size")
#             print(x_l.size())
            # curr time step 
            new_h_l = self.gru_l(self.squeeze_d2(x_l[:,:,i:i+1,:]), x_mean_l[i:i+1,:], prev_h_l)
            new_h_a = self.gru_a(self.squeeze_d2(x_a[:,:,i:i+1,:]), x_mean_a[i:i+1,:], prev_h_a)
            # curr time step
#             new_h_l, new_c_l = self.lstm_l(x_l[i], (self.h_l, self.c_l))
#             new_h_a, new_c_a = self.lstm_a(x_a[i], (self.h_a, self.c_a))
   
            # concatenate
            prev_hs = torch.cat([prev_h_l,prev_h_a], dim=1)
            new_hs = torch.cat([new_h_l,new_h_a], dim=1)
            
            cStar = torch.cat([prev_hs,new_hs], dim=1)
            attention = F.softmax(self.att1_fc2(self.att1_dropout(F.relu(self.att1_fc1(cStar)))),dim=1)
            attended = attention*cStar
            
            cHat = F.tanh(self.att2_fc2(self.att2_dropout(F.relu(self.att2_fc1(attended)))))
            
            both = torch.cat([attended,self.mem], dim=1)
            gamma1 = F.sigmoid(self.gamma1_fc2(self.gamma1_dropout(F.relu(self.gamma1_fc1(both)))))
#             gamma2 = F.sigmoid(self.gamma2_fc2(self.gamma2_dropout(F.relu(self.gamma2_fc1(both)))))
        
            self.mem = gamma1*self.mem + (1-gamma1)*cHat
        
            # decay teh memory according to delta
            gamma_decay = F.sigmoid(self.map_gamma_decay(F.relu(self.gamma_decay(torch.squeeze(x[:,3,i:i+1,:], dim=1)))))
            self.mem = self.mem * gamma_decay 
        
            # record delta condition for memory 
            
            all_mems.append(self.mem)
            # update
            self.h_l = new_h_l
            self.h_a = new_h_a

            all_h_ls.append(self.h_l)
            all_h_as.append(self.h_a)

        # last hidden layer last_hs is n x h
        last_h_l = all_h_ls[-1]
        last_h_a = all_h_as[-1]

        last_mem = all_mems[-1]
        last_hs = torch.cat([last_h_l,last_h_a,last_mem], dim=1)
        output = self.out_fc2(self.out_dropout(F.relu(self.out_fc1(last_hs))))
        return output
    
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State

In [18]:
config = dict()
config["input_dims"] = [1, 4]
hl = 32
ha = 32
config["h_dims"] = [hl, ha]
config["memsize"] = 64
config["windowsize"] = 2
config["batchsize"] = 32
config["num_epochs"] = 50
config["lr"] = 0.0001
NN1Config = dict()
NN1Config["shapes"] = 32
NN1Config["drop"] = 0.7
NN2Config = dict()
NN2Config["shapes"] = 32
NN2Config["drop"] = 0.7
gamma1Config = dict()
gamma1Config["shapes"] = 32
gamma1Config["drop"] = 0.7
gamma2Config = dict()
gamma2Config["shapes"] = 32 
gamma2Config["drop"] = 0.7
outConfig = dict() 
outConfig["shapes"] = 32
outConfig["drop"] = 0.7
configs = [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig]

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_mfn(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, configs, x_mean_aft_nor)
# train_grud(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, config, x_mean_aft_nor)

epoch train_loss valid_loss


/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0 141.68802025204613 110.31683349609375 saving model


/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type MFN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type GRUD. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type FilterLinear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


1 140.92018272763207 109.72466278076172 saving model
2 140.36889721098402 109.13855743408203 saving model
3 139.5702641805013 108.5344467163086 saving model
4 138.96051497686477 107.8864974975586 saving model
5 138.28738839285714 107.1755599975586 saving model
6 137.61962817964098 106.3686752319336 saving model
7 136.3516547793434 105.39273071289062 saving model
8 135.14534541538782 104.18343353271484 saving model
9 133.7677045549665 102.67984771728516 saving model
10 131.62289356050036 100.7883071899414 saving model
11 129.52580224900018 98.53624725341797 saving model
12 127.26117851620629 95.83786010742188 saving model
13 123.18883223760696 92.4926986694336 saving model
14 119.68937283470517 88.4238510131836 saving model
15 114.09637142363049 83.46207427978516 saving model
16 109.2572386605399 77.57197570800781 saving model
17 102.2149189540318 70.50241088867188 saving model
18 94.00424266996838 62.5827522277832 saving model
19 86.73120807466053 54.1258659362793 saving model
20 77.35

In [2]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_mfn(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, configs, x_mean_aft_nor)
# train_grud(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, config, x_mean_aft_nor)

NameError: name 'np' is not defined

In [3]:
x_mean_aft_nor.shape

NameError: name 'x_mean_aft_nor' is not defined

In [4]:
config = dict()
config["input_dims"] = [1, 4]
hl = 128
ha = 128
config["h_dims"] = [hl, ha]
config["memsize"] = 128
config["windowsize"] = 2
config["batchsize"] = 128
config["num_epochs"] = 100
config["lr"] = 0.001
NN1Config = dict()
NN1Config["shapes"] = 128
NN1Config["drop"] = 0.7
NN2Config = dict()
NN2Config["shapes"] = 128
NN2Config["drop"] = 0.7
gamma1Config = dict()
gamma1Config["shapes"] = 128
gamma1Config["drop"] = 0.7
gamma2Config = dict()
gamma2Config["shapes"] = 128 
gamma2Config["drop"] = 0.7
outConfig = dict() 
outConfig["shapes"] = 128
outConfig["drop"] = 0.7
configs = [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig]

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_mfn(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, configs, x_mean_aft_nor)
# train_grud(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, config, x_mean_aft_nor)

NameError: name 'np' is not defined

In [5]:
config = dict()
config["input_dims"] = [1, 4]
hl = 32
ha = 32
config["h_dims"] = [hl, ha]
config["memsize"] = 64
config["windowsize"] = 2
config["batchsize"] = 32
config["num_epochs"] = 50
config["lr"] = 0.0001
NN1Config = dict()
NN1Config["shapes"] = 32
NN1Config["drop"] = 0.7
NN2Config = dict()
NN2Config["shapes"] = 32
NN2Config["drop"] = 0.7
gamma1Config = dict()
gamma1Config["shapes"] = 32
gamma1Config["drop"] = 0.7
gamma2Config = dict()
gamma2Config["shapes"] = 32 
gamma2Config["drop"] = 0.7
outConfig = dict() 
outConfig["shapes"] = 32
outConfig["drop"] = 0.7
configs = [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig]

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_mfn(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, configs, x_mean_aft_nor)
# train_grud(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, config, x_mean_aft_nor)

NameError: name 'np' is not defined

In [ ]:
[a, b] = config["input_dims"]

In [ ]:
a